<br><br><br>
<h1><font color="#B30033" size=5>Sistemas Inteligentes - Curso 2023-2024</font></h1>



<h1><font color="#B30033" size=5>Laboratorio 2: Aprendizaje por refuerzo</font></h1>


<br>
<div estilo="text-align: izquierda">
<font color="#4E70BE" size=3>Profesores:</font><br>
<ul>
<li><font color="#4E70BE" size=3>Juan Carlos Alfaro Jiménez (JuanCarlos.Alfaro@uclm.es)</font><br></li>
<li><font color="#4E70BE" size=3>Mª Julia Flores Gallego (Julia.Flores@uclm.es)</font><br></li>
<li><font color="#4E70BE" size=3> Ismael García Varea (Ismael.Garcia@uclm.es)</font><br></li>
<li><font color="#4E70BE" size=3> Luis González Naharro (Luis.GNaharro@uclm.es)</fuente>
<li><font color="#4E70BE" size=3> Luna Jimenéz Fernández (Luna.Jimenez@uclm.es)</font><br></li>
<li><font color="#4E70BE" size=3> Marina Sokolova (Marina.Sokolova@uclm.es)</font>
<br></li>
</ul>
</div>

# ¡Rescate Urbano!

### (Versión para RL)




## 1. Introducción


En esta práctica vamos a continuar con el mismo dominio que en la práctica 1: el rescate urbano. Muchos de los elementos que utilizamos en práctica 1 se mantienen, aunque hemos introducido algunas novedades para adaptarlo a la nueva familia de algoritmos. En concreto:

- Una casilla con una persona atrapada se considera un estado terminal.
- Cada persona atrapada tendrá asociado un valor, que básicamente es la recompensa de rescatar a esa persona.
- Habrá peligros como en la práctica 1 con recompensa -5.0, pero podrían existir peligros finales con recompensas negativas mayores, cuyos valores se especificarán en la definición del problema.
- El entorno es estocástico, tal y como hemos visto en el Tema 5. Por ejemplo, si el agente decide hacer la acción 'UP', con una probabilidad de $p$ terminará en la posición esperada, mientras que con una probabilidad $(1-p)/2$ a cada una de las direcciones perpendiculares al movimiento deseado. Por ejemplo, si $p = 0.6$, el agente se encuentra en la casilla (2,3) y la acción es "RIGHT", suponiendo que (1,3), (2,4) y (3,3) no están bloqueadas, 60% de las veces el agente se moverá a la casilla (2,4), el 20% a la (1,3) y otro 20% a la (3,3).

En esta práctica, por tanto, el robot de rescate tendrá que calcular la política óptima que maximice la recompensa final. Dado que rescatar a una persona consiste en alcanzar un estado final, aun habiendo que realizar varios rescates, la tarea será similar a rescatar a **una sola persona**, teniendo en cuenta que algunos rescates serán más rentables que otros.

Como en la práctica 1, la ciudad está representada por una cuadrícula donde cada celda puede tener diferentes estados: bloqueada, libre, con personas atrapadas (con distintas recompensas) o con peligros (algunos de ellos fatales).

### Objetivos:

- Aplicar el algoritmo Q-Learning para el aprendizaje por refuerzo para encontrar la mejor política para maximizar la recompensa a largo plazo.
- Implementar al menos una estrategia de exploración.
- Realizar un análisis del rendimiento del algoritmo propuesto, considerando los diferentes parámetros: $p$ (probabilidad de alcanzar la posición deseada, perteneciente a un entorno estocástico), $r$ (la recompensa inmediata de los estados no finales y no peligrosos), $\alpha$ (la tasa de aprendizaje), $\gamma$ (factor de descuento), y $\epsilon$ (probabilidad de exploración-explotación).

En este trabajo pondremos en práctica las técnicas de aprendizaje por refuerzo vistas en clase. También analizaremos y compararemos el rendimiento del algoritmo implementado, en función de la parametrización y considerando diferentes instancias del problema.

Esperamos que esta práctica os ayude a profundizar la comprensión del aprendizaje por refuerzo, su propósito y os anime a pensar en cómo se pueden aplicar estas técnicas en situaciones del mundo real para ayudar en operaciones de rescate y otras tareas críticas. ¡Buena suerte!


## 2. Descripción del problema

Se trata de resolver un problema en el que un robot tiene que encontrar la mejor estrategia en una situación de rescate de una sola persona, tratando de evitar obstáculos y zonas peligrosas, y sin correr peligros fatales. El espacio de búsqueda está definido por una cuadrícula donde el robot puede moverse en cuatro direcciones: UP, RIGHT, DOWN y LEFT.

Más formalmente, el problema se puede definir como:
- Estados:
    - Libres: casillas transitables por las que se puede pasar sin problemas.
    - Bloqueado: casillas intransitables debido a obstáculos como edificios, paredes, vallas, etc.
    - Personas atrapadas: ubicaciones donde se encuentran grupos de personas que necesitan ser rescatadas, en este caso cada una de ellas tendrá un valor asociado (la recompensa de este rescate).
    - Peligros: áreas con peligros adicionales que pueden retrasar el rescate o incrementar el costo de la ruta (por ejemplo, incendios, áreas inestables, etc.).
    - Peligros Fatales: Peligros que no se pueden superar, una vez que el robot se mete en uno de ellos que averiado o atrapado, lo cual tendrá un coste distinto según el caso concreto.
    - Estado inicial: es el lugar desde donde el robot iniciará cada rescate (y por tanto cada episodio), pero esto es algo que el algoritmo podría cambiar, si así nos piden.
    - Estado final: casilla con una o varias personas atrapadas y por tanto con recompensa positiva, o bien un peligro fatal y por tanto con una recompensa negativa.
- Acciones: posibles movimientos del robot de rescate UP, RIGHT, DOWN, LEFT) desde la celda actual.

El entorno será estocástico, como se explicó anteriormente en la introducción. Por tanto, dado un estado $s$ y una acción $a$, el estado resultante podría ser diferente en función de la estocasiticidad del entorno.

En la carpeta './initial-rl-instances' hay algunos ejemplos de instacias de problemas a resolver.

## 3. Desarrollo de la práctica

Durante el desarrollo de la práctica facilitaremos un conjunto de instancias de problemas a resolver. La dimensionalidad, tanto en el número como en el tipo de celdas, será variable, y los algoritmos implementados deberán ser lo suficientemente eficientes para funcionar correctamente en todas ellas. Algunos otros escenarios se mantendrán para la evaluación/corrección/entrevista, por tanto es importante desarrollar el código de manera que sea lo suficientemente general como para poder resolverlos fácilmente.

### 3.1 Problemas de entrada

Cada escenario tendrá asociado un archivo JSON con la siguiente estructura:

```JSON
{
    "city": {
        "rows": number_of_rows,
        "columns": number_of_columns,
        "blocked": [[x_b1,y_b1],...,[x_bn,y_bn]]
        },
    "departure": [x_init,y_init],            
    "dangers": [[x_d1,y_d1],...,[x_dk, y_dk]],
    "trapped": [[x_t1,y_t1,r_t1],...,[x_tm, y_tm, r_tm]],
    "fatal_dangers": [[x_f1,y_f1,r_f1],...,[x_fz, y_fz, r_fz]],
}
```

Hay cinco claves generales en el archivo JSON:
- `city`: diccionario que representa el escenario del problema.
- `departure`: casilla de salida, es decir, la ubicación inicial del robot.
- `dangers`: lista con las celdas donde hay un peligro.
- `trapped`: lista con las celdas donde hay personas atrapadas y el robot tiene que ir a rescatarlas, junto con el valor de la recompensa asociada a ese rescate. Por ejemplo, [3,2,100] signifiva que la persona a rescatar está en la casilla (x=3,y=2) y la recompensa asociada es 100.
- `fatal_dangers`: lista con las celdas donde se encuentran peligros fatales junto con  el valor de la recompensa **negativa** asociada. Por ejemplo, [7,4,-50] significa que el peligro fatal está en la casilla (x=7,y=4) y la recompensa asociada es -50.

En el diccionario `city`, hay tres claves, al igual que en la práctica 1:
- `rows`: número de filas de la cuadrícula.
- `columns`: número de columnas de la grilla.
- `blocked`: lista con las celdas por donde el robot no puede pasar.

Cada celda de la cuadrícula se define como un par (x,y) (que en formato json se representa como una lista de dos números [x,y]):
- `x`: la coordenada x de la celda o la fila de la cuadrícula.
- `y`: la coordenada y de la celda o la columna de la cuadrícula.

Asumimos que la celda de la esquina superior izquierda es (0,0), y para una celda determinada (x,y), se considera que el eje x crece hacia abajo (filas) y el eje y crece de izquierda a derecha (columnas). Por ejemplo, en la imagen dada el estado inicial sería (6,1). Y la esquina inferior derecha será (9,9), la esquina inferior izquierda será (9,0) y la esquina superior derecha será (0,9).

Un posible ejemplo de este problema podría ser el que se muestra aquí:

<img src="./Figure_1.png" alt="Simple example" width="500" height="500"/>

NOTA: En otras aplicaciones (procesamiento de imágenes) estos ejes se intercambian pero este sistema facilitará su trabajo al utilizar listas 2D (matrices) en Python.

## 4. Plan de trabajo

En esta prácticas se deberán realizar las siguientes tareas:

- Diseño del Entorno:
    - Involucra a la ciudad y los elementos que la componen.
    - Implica también la interacción con el agente: determinación de la recompensa y del nuevo estado, una vez que recibe una acción del agente.

- Diseño del algoritmo de aprendizaje por refuerzo:
    - Describir el espacio de estados, e identificarlos en una instancia concreta.
    - Describir las acciones.
    - Describir las recompensas.
    - Distinguir entre estado intermedio y terminal.

- Implementación del Algoritmo Q-Learning:
    - Inicialización de la Q-Tabla.
    - Manejar los parámetros necesarios del algoritmo (alfa, gamma, épsilon, etc.).
    - Regla de actualización.
    - Ejecución de un episodio, que al mismo tiempo, implicará la ejecución de transiciones individuales.
    - Implementar un mecanismo de exploración.
    - Extraer la política de la Q-tabla y actuar en consecuencia a esa política.

- Experimentación y Análisis:
    - Analizar el rendimiento del algoritmo implementado en función de los parámetros y la instancia del problema.
    - Comparar y contrastar los resultados obtenidos de las diferentes parametrizaciones e instancias.

- Memoria:
    - Redactar una memoria detallando el proceso seguido, la definición del entorno, el algoritmo implementado, el rendimiento en función de las configuraciones elegidas y los resultados obtenidos.

En la evaluación del trabajo tendremos en cuenta:
- Correcta implementación del entorno y algoritmo Q-learning: 50%.
- Diseño y uso adecuado de los argumentos, e inclusión exitosa de la exploración: 20%.
- Experimentación y análisis de resultados: 20%.
- Calidad del informe: 10%.

Fechas de presentación de la práctica:
- Fecha límite para enviar el código (libreta): **Viernes 15 de diciembre de 2023**.
- Fecha límite de presentación del informe: **Viernes 22 de diciembre de 2023**.

A continuación se proporcionan más detalles de cada tarea.

### 4.1 Formalización del problema y ejemplos

En primer lugar, este problema de rescate urbano con recompensas debe formalizarse como un problema de aprendizaje por refuerzo: estados, acciones, políticas, etc., definiendo sus elementos básicos. También se debe distinguir el entorno del algoritmo de aprendizaje, teniendo el entorno también elementos importantes: estado, modelo de transición y modelo de recompensa, ya que el entorno proporciona al agente la recompensa obtenida ($r$) y el estado resultante ($s'$), tal y como hemos visto en el tema 5. Todas las implementaciones deben incluir algún mecanismo de exploración y la posibilidad de cambiar fácilmente los distintos argumentos involucrados tanto en el algoritmo como en el entorno. También se debe poder extraer (y dibujar) de la Q-tabla la política que el algoritmo ha aprendido.

### 4.2 Implementación

A continuación, se debe proporcionar el código Python que resuelva la práctica. Debéis codificar vuestra propia estructura de clases: el entorno y el algoritmo de aprendizaje, con al menos los componentes más esenciales:
- para el algoritmo: tasa de aprendizaje, factor de descuento, parámetro de exploración (modificable), extracción de la política aprendida.
- para el entorno: distribución de probabilidad del grado de estocasticidad.

### 4.3 Estudio y mejora de los algoritmos
Una vez implementado el entorno y el algoritmo, se debe estudiar su comportamiento, considerando un buen estudio/análisis sobre los distintos parámetros, y también en función de la dimensionalidad y complejidad del escenario.

Para ello se debe estudiar la convergencia del algoritmo, la calidad de las políticas obtenidas, la influencia de los distintos valores de los argumentos en el $Q(s,a)$ aprendido. Su implementación debe optimizar el código tanto en tiempo de computación como en asignación de memoria. También son importantes factores como el tamaño máximo del problema que se puede resolver (sin desbordamiento de memoria) o el efecto de utilizar escenarios más complejos. Además, se pueden proponer implementaciones alternativas que aumenten la eficiencia de los algoritmos.

### 4.4 Memoria
Además de la libreta que contiene la implementación, el trabajo consiste en la elaboración de una memoria, que tendrá como fecha de entrega límite el final del semestre docente, junto con la memoria de la práctica 1.

En particular, entre otros temas que el estudiante considere de interés para discutir, el la memoria deberá incluir al menos:
- Una breve descripción del problema, una descripción de cómo se ha implementado el algoritmo, la evaluación del desempeño y la descripción de las mejoras si existen.
- El efecto de los distintos parámetros tanto en el entorno como en el algoritmo Q-learning.
- Debe proporcionarse al menos un mecanismo de exploración.
- Además de su descripción y motivación, se debe incluir un análisis donde se estudie la convergencia de valores $Q(s,a)$.
- Es necesaria poder obtener una política a partir de la Q-tabla.

El informe no debe incluir figuras con código fuente, a menos que sea necesario para explicar algún concepto clave (estructuras de datos, mejoras de eficiencia, etc.). En tales casos, se permite incluir pseudocódigo con el formato adecuado.

## 5. Presentación y evaluación
Es muy recomendable realizar el trabajo en grupo (por parejas), aunque se puede realizar de forma individual. La fecha límite de presentación es el 15 de diciembre de 2023. Los exámenes orales (entrevistas) para la evaluación se realizarán la semana siguiente y de forma individual.

Algunas consideraciones relacionadas con la evaluación:
- A esta práctica le corresponde el 70% de la nota de práctica. La práctica 1 (30%) debe haberse entregado y evaluado para aprobar la asignatura. Los envíos fuera de plazo (junto con la práctica 2) o en caso de haber suspendido se evaluarán globalmente pero con una penalización del 10%, por lo que los estudiantes sólo podrán obtener el 90% de la nota de laboratorio si se da el caso.
- La asistencia a las prácticas no es obligatoria, pero será la mejor base para la resolución exitosa de las mismas.
- Recordar que las dudas y preguntas sobre las prácticas de laboratorio deben resolverse principalmente en las sesiones de laboratorio.
- El trabajo será evaluado mediante una entrevista individual (examen) con los profesores. Las fechas de las entrevistas se publicarán con cierta antelación en Campus Virtual.
- Proporcionaremos un conjunto de casos de prueba preliminares (varios escenarios) que deben resolverse correctamente. En caso contrario, la práctica no se considerará aprobada.
- Es obligatorio realizar la entrevista y responder, de forma individual, a una serie de preguntas sobre la organización del código y sobre cuestiones relacionadas.
- En la evaluación no continua requeriremos además la implementación del Algoritmo de Iteración de Políticas. También se pueden requerir características adicionales.